In [24]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
print("Setup Complete")
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

Setup Complete


In [25]:
def classificate(value):
    target='neutral'
    if(value>=0.001 and value<=0.002):
        target='buy'
    elif(value>=0.002):
        target='strong buy'
    elif(value<=-0.001 and value>=-0.002):
        target='sell'
    elif(value<-0.002):
        target='strong sell'
    return target


def classificateWithNumbers(value):
    target=0
    if(value>=0.001 and value<=0.002):
        target=1
    elif(value>=0.002):
        target=2
    elif(value<=-0.001 and value>=-0.002):
        target=-1
    elif(value<-0.002):
        target=-2
    return target

def classificateWithNumbersThreeTarget(value):
    target=0
    if(value>=0.0015):
        target=1
    elif(value<=-0.0015):
        target=-1
    else:
        target=0
    return target



def classificateWithIncreasingNumbers(value):
    target=3
    if(value>=0.0015 and value<=0.0025):
        target=4
    elif(value>=0.0025):
        target=5
    elif(value<=-0.0015 and value>=-0.0025):
        target=2
    elif(value<-0.0025):
        target=1
    return target

def correlation_heatmap(df):
    correlations = df.corr()

    fig, ax = plt.subplots(figsize=(40,40))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.1, annot=True, cbar_kws={"shrink": .7})
    sns.set(font_scale=2)
    plt.show();

def aboveValue(openList,inspectList,df,name):
    results=[]
    for i in range(len(inspectList)):
        if (openList[i]>inspectList[i]):
            results.append(1)
        else:
            results.append(0)
    df[name]=results
    

In [26]:
data1 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_06', sep=",", header=None)
data1.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data2 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_07', sep=",", header=None)
data2.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data3 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_08', sep=",", header=None)
data3.columns = ["time", "open", "high", "low","close","volume",'drop1','drop2']

data4 = pd.read_csv('/home/balazs/Desktop/adathalmazok/btc_2012-2021/btc_split_data_part_09', sep=",", header=None)
data4.columns = ["time", "open", "high", "low","close","volume"]

data= pd.concat([data1,data2,data3,data4], axis=0)


#data=data4

In [27]:
%%time
#btc_data = pd.read_csv(r"/home/balazs/Desktop/szakdoga_info/btc_data.csv")
#data = btc_data.dropna()
data['MA5']=data['open'].rolling(window =5).mean()
data['EMA5']=data['open'].ewm(span=5,min_periods=0,adjust=False,ignore_na=False).mean()
data['CMA']=data.open.expanding().mean()
data['open_percentage_change']=data.open.pct_change()
data['close_percentage_change']=data.close.pct_change()
data['high_percentage_change']=data.high.pct_change()
data['low_percentage_change']=data.low.pct_change()
data['volume_percentage_change']=data.volume.pct_change()
lista=[]
for i in range(len(data.open_percentage_change.values)-5):
        value=data.open_percentage_change.values[i+1]+data.open_percentage_change.values[i+2]+data.open_percentage_change.values[i+3]+data.open_percentage_change.values[i+4]+data.open_percentage_change.values[i+5]
        lista.append(value)
for i in range(5):
    lista.append(0)
data['profit']=lista
buy_sell_list=[classificateWithNumbersThreeTarget(elem) for elem in data.profit]
data['decision']=buy_sell_list

CPU times: user 44.9 s, sys: 156 ms, total: 45.1 s
Wall time: 45.2 s


In [28]:
data=data.dropna()

In [29]:
data.drop(data.tail(5).index,inplace=True) # drop last n rows

In [30]:
aboveValue(data.open.values,data.MA5.values,data,'aboveMA5')
aboveValue(data.open.values,data.EMA5.values,data,'aboveEMA5')
aboveValue(data.open.values,data.CMA.values,data,'aboveCMA')

In [31]:
X=data[['close_percentage_change','high_percentage_change','low_percentage_change']]
y=data[['decision']]

In [32]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [33]:
from imblearn.under_sampling import TomekLinks
TK = TomekLinks(sampling_strategy='majority')
X, y = TK.fit_resample(X, y)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [35]:
len(X_train)

1783122

In [36]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier



# instantiate the classifier 

rfc = RandomForestClassifier(random_state=0)



# fit the model

rfc.fit(X_train, y_train)



# Predict the Test set results

y_pred = rfc.predict(X_test)


In [37]:
from sklearn.metrics import classification_report
classes = ['-1','0','1']
print(classification_report(y_test, y_pred, target_names = classes, zero_division = 0))

              precision    recall  f1-score   support

          -1       0.77      0.46      0.57    240274
           0       0.47      0.83      0.60    261716
           1       0.76      0.46      0.57    262206

    accuracy                           0.59    764196
   macro avg       0.67      0.58      0.58    764196
weighted avg       0.66      0.59      0.58    764196



In [42]:
X_test

,close_percentage_change,high_percentage_change,low_percentage_change
103499,-0.000067,-3.035658e-03,-0.001271
1393635,-0.000760,7.775333e-07,0.000103
1574378,-0.001677,-2.345760e-04,0.000211
1233790,-0.001387,-2.307611e-04,-0.001171
1710690,-0.001347,1.399488e-03,0.001581
...,...,...,...
376861,0.001288,1.287650e-03,0.000811
777919,0.000427,5.351456e-04,0.000427
1025465,-0.000569,-2.298892e-04,-0.000151
1407225,-0.013070,-4.818818e-03,-0.012730


In [49]:
%time
rfc.predict([[0.001,0.000
              ,0.000]])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


array([0])

In [47]:
import joblib
joblib.dump(rfc,'randomForest.joblib')

['randomForest.joblib']